In [7]:
from pymongo import MongoClient, InsertOne, UpdateOne, DeleteOne

client = MongoClient('mongodb://localhost:27017/')
db = client['university_db']
courses_collection = db['courses']

# Bulk inserto of course with student enrollments
operations = [ 
    InsertOne({'course': 'Math 101', 'enrollments':30, 'department': 'Mathematicts'}), 
    InsertOne({'course': 'CS 102', 'enrollments':25, 'department': 'Computer Science'}),
    InsertOne({'course': 'History 201', 'enrollments':20, 'department': 'History'}),
    InsertOne({'course': 'Physics 202', 'enrollments':15, 'department': 'Physics'})
]
courses_collection.bulk_write(operations)
print("Courses inserted successfully.")

Courses inserted successfully.


In [8]:
### 2. Complex Filtering and Querying 

# Query for courses with enrollments greater than 20 
for course in courses_collection.find({'enrollments': {'$gt': 20}}):
    print(course)

{'_id': ObjectId('693c0f1f625ce337af59e4b8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematicts'}
{'_id': ObjectId('693c0f1f625ce337af59e4b9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [9]:
# Query courses in Computer Science or Mathematics departments 
for course in courses_collection.find({'department': {'$in': ['Computer Science', 'Mathematicts']}}):
        print(course)

{'_id': ObjectId('693c0f1f625ce337af59e4b8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematicts'}
{'_id': ObjectId('693c0f1f625ce337af59e4b9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science'}


In [10]:
### 3.Aggregation Framework for Data Analysis

# Average enrollment per department using aggregation
pipeline = [ 
    {'$group': {'_id': '$department', 'average_enrollment': {'$avg': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline): 
    print(result)

{'_id': 'History', 'average_enrollment': 20.0}
{'_id': 'Mathematicts', 'average_enrollment': 30.0}
{'_id': 'Physics', 'average_enrollment': 15.0}
{'_id': 'Computer Science', 'average_enrollment': 25.0}


In [11]:
# Maximum enrollment per department 
pipeline = [
    {'$group': {'_id': '$department', 'max_enrollment': {'$max': '$enrollments'}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Mathematicts', 'max_enrollment': 30}
{'_id': 'History', 'max_enrollment': 20}
{'_id': 'Computer Science', 'max_enrollment': 25}
{'_id': 'Physics', 'max_enrollment': 15}


In [12]:
### 4.Data Transformation using projectandaddFields

# Projection to rename fields 
pipeline = [
    {'$project': {'course_name': '$coures', 'department_name': '$department', 'enrollments':1}}
]
for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('693c0f1f625ce337af59e4b8'), 'enrollments': 30, 'department_name': 'Mathematicts'}
{'_id': ObjectId('693c0f1f625ce337af59e4b9'), 'enrollments': 25, 'department_name': 'Computer Science'}
{'_id': ObjectId('693c0f1f625ce337af59e4ba'), 'enrollments': 20, 'department_name': 'History'}
{'_id': ObjectId('693c0f1f625ce337af59e4bb'), 'enrollments': 15, 'department_name': 'Physics'}


In [13]:
# Adding enrollments category field based on enrollments 

pipeline = [
    {'$addFields': {'enrollment_category': {'cond': {'if': {'$gt': ['enrollments', 20]}, 'then': 'high', 'else': 'low'}}}}
]
for result in courses_collection.aggregate(pipeline): 
    print(result) 

{'_id': ObjectId('693c0f1f625ce337af59e4b8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematicts', 'enrollment_category': {'cond': {'if': True, 'then': 'high', 'else': 'low'}}}
{'_id': ObjectId('693c0f1f625ce337af59e4b9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'enrollment_category': {'cond': {'if': True, 'then': 'high', 'else': 'low'}}}
{'_id': ObjectId('693c0f1f625ce337af59e4ba'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'enrollment_category': {'cond': {'if': True, 'then': 'high', 'else': 'low'}}}
{'_id': ObjectId('693c0f1f625ce337af59e4bb'), 'course': 'Physics 202', 'enrollments': 15, 'department': 'Physics', 'enrollment_category': {'cond': {'if': True, 'then': 'high', 'else': 'low'}}}


In [14]:
### Homework 
# 1. Hitung jumlah course per department 

pipeline = [
    {'$group': {'_id': '$department', 'total_courses': {'$sum': 1}}}
]
for result in courses_collection.aggregate(pipeline):
    print(result) 

{'_id': 'Mathematicts', 'total_courses': 1}
{'_id': 'History', 'total_courses': 1}
{'_id': 'Computer Science', 'total_courses': 1}
{'_id': 'Physics', 'total_courses': 1}


In [15]:
# 2. Filter course dengan $match dan $group 
pipeline = [
    {'$match': {'department': 'Computer Science', 'enrollments': {'$gte': 25}}},
    {'$group': {'_id': '$department', 'courses': {'$push': '$course'}}}
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': 'Computer Science', 'courses': ['CS 102']}


In [16]:
# 3.Menggabungkan koleksi dengan $lookup

students_collection = db['students']

students_collection.insert_many([
    {'name': 'Alice', 'course': 'CS 102'},
    {'name': 'Budi', 'course': 'Math 101'},
    {'name': 'Cindy', 'course': 'History 201'}
])

pipeline = [
    {
        '$lookup': {
            'from': 'students',          # koleksi yang akan digabung
            'localField': 'course',      # field di koleksi 'courses'
            'foreignField': 'course',    # field di koleksi 'students'
            'as': 'student_details'      # hasil join disimpan di field baru
        }
    }
]

for result in courses_collection.aggregate(pipeline):
    print(result)

{'_id': ObjectId('693c0f1f625ce337af59e4b8'), 'course': 'Math 101', 'enrollments': 30, 'department': 'Mathematicts', 'student_details': [{'_id': ObjectId('69129225a7867c2dd551733e'), 'name': 'Budi', 'course': 'Math 101'}, {'_id': ObjectId('693c0f39625ce337af59e4bd'), 'name': 'Budi', 'course': 'Math 101'}]}
{'_id': ObjectId('693c0f1f625ce337af59e4b9'), 'course': 'CS 102', 'enrollments': 25, 'department': 'Computer Science', 'student_details': [{'_id': ObjectId('69129225a7867c2dd551733d'), 'name': 'Alice', 'course': 'CS 102'}, {'_id': ObjectId('693c0f39625ce337af59e4bc'), 'name': 'Alice', 'course': 'CS 102'}]}
{'_id': ObjectId('693c0f1f625ce337af59e4ba'), 'course': 'History 201', 'enrollments': 20, 'department': 'History', 'student_details': [{'_id': ObjectId('69129225a7867c2dd551733f'), 'name': 'Cindy', 'course': 'History 201'}, {'_id': ObjectId('693c0f39625ce337af59e4be'), 'name': 'Cindy', 'course': 'History 201'}]}
{'_id': ObjectId('693c0f1f625ce337af59e4bb'), 'course': 'Physics 202',